Code for scraping from indeed.co.uk

In [1]:
#import packages
import requests, bs4, time
import pandas as pd
import os
import scraping_funcs_indeeduk as sf
from itertools import cycle
import traceback
from datetime import date
import pickle


In [2]:
ads= pd.DataFrame(columns = ['company', 'job_title', 'salary', 'location', 'description', 'date',
       'full_description', 'other_deets', 'jobtype', 'salary_low',
       'salary_high','extraction_date'])

import os

for i in os.listdir('C:/Users/lundr/DataScienceJobs/data/indeed_raw_pickles/'):
    
    read_in = pd.read_pickle("C:/Users/lundr/DataScienceJobs/data/indeed_raw_pickles/"+i)
    ads = pd.concat([ads,read_in],ignore_index = True, axis = 0, sort=True)


In [9]:
ads['days_from_post']  = 'None'

for i in range(ads.shape[0]):
    if (ads['date'][i] == 'Today')|(ads['date'][i] == 'Just posted') :
        ads['days_from_post'] = 0
    elif ads['date'][i] == 'Nothing_found':
        pass
    else:
        a = ads['days_from_post'][i] = ads['date'][i].split("d",1)[0]
        ads['days_from_post'][i] = a.split("+",1)[0]


C:\Users\lundr\AppData\Local\Continuum\anaconda3\envs\dsj\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\lundr\AppData\Local\Continuum\anaconda3\envs\dsj\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\lundr\AppData\Local\Continuum\anaconda3\envs\dsj\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydat

In [6]:
#ads['extraction_date'] = pd.to_datetime(ads['extraction_date'])
import datetime
ads['posted_date']='None'
for x in range(len(ads['days_from_post'])):
    if ads['days_from_post'][x] is not 'None':
        ads['posted_date'][x] = pd.to_datetime(ads['extraction_date'][x])  - datetime.timedelta(days=int((ads['days_from_post'][x])))      




C:\Users\lundr\AppData\Local\Continuum\anaconda3\envs\dsj\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [114]:
 ads['posted_date']

0       2019-06-11 00:00:00
1       2019-06-11 00:00:00
2       2019-06-11 00:00:00
3       2019-06-11 00:00:00
4       2019-06-11 00:00:00
               ...         
4510    2019-10-12 00:00:00
4511    2019-11-06 00:00:00
4512    2019-10-12 00:00:00
4513    2019-10-27 00:00:00
4514    2019-10-27 00:00:00
Name: posted_date, Length: 4515, dtype: object

In [10]:
ads.head()

,index,company,date,description,extraction_date,full_description,job_title,jobtype,location,other_deets,salary,salary_high,salary_low,days_from_post,posted_date,salary_type
0,0,\nSingle Homelessness Prevention Service,Today,\n Data and Operations Analyst with an innovat...,6-11-2019,Role: Data and Operations Analyst with an inno...,Homelessness Data and Operations Analyst,NaN,Nothing_found,"London_£33,000 a year","\n£33,000 a year","\n£33,000 a year","\n£33,000 a year",0,None,year
1,1,\n\nHalfords,Nothing_found,\n Previous experience of managing data and re...,6-11-2019,As Operations Analyst for Halfords you'll be r...,Data Analyst - Operations,NaN,Nothing_found,Worcestershire,Nothing_found,Nothing_found,Nothing_found,0,None,None
2,2,\n\nBank of America,Nothing_found,\n Operations Analyst – OTC Clearing Settlemen...,6-11-2019,Operations Analyst – OTC Clearing Settlements\...,Operations Analyst,NaN,Nothing_found,Chester,Nothing_found,Nothing_found,Nothing_found,0,None,None
3,3,\n\nMaritzCX,Today,\n 0-2 years’ experience in data processing or...,6-11-2019,"Description: Learn the root techniques, Survox...",Data Collection Analyst,NaN,Nothing_found,"High Wycombe_Full-time, Permanent_£23,000 - £2...","\n£23,000 - £27,000 a year","£27,000 a year","\n£23,000",0,None,year
4,4,\nValpak Ltd,Nothing_found,\n Compile data as part of Consulting projects...,6-11-2019,As the EPIC Analyst you will primarily support...,Data Analyst (EPIC),NaN,Nothing_found,"Stratford-upon-Avon_Full-time, Permanent",Nothing_found,Nothing_found,Nothing_found,0,None,None


In [44]:
# clean data
import cleaning_funcs_indeed_uk as cf
ads['location'],ads['salary'],ads['jobtype'] = cf.clean_other_deets(ads['other_deets'])
ads['full_description'] = cf.clean_column(ads['full_description'])
ads['company'] = cf.clean_column(ads['company'])
ads['salary_low'], ads['salary_high'],ads['salary_type'] = cf.create_split_salary_range(ads['salary'])

########
ads['salary_low']=cf.clean_salary(ads['salary_low'],"£")
ads['salary_high']=cf.clean_salary(ads['salary_high'],"£")


In [7]:
import cleaning_funcs_indeed_uk as cf
ads['salary_low'], ads['salary_high'],ads['salary_type'] = cf.create_split_salary_range(ads['salary'])

In [45]:
ads = ads.drop_duplicates(['description', 'other_deets'])

In [62]:
companies = list(ads.company)

In [63]:
print(companies)

[' Single Homelessness Prevention Service', '  Halfords', '  Bank of America', '  MaritzCX', ' Valpak Ltd', '  MDSL', ' MI ', ' Caresoft', '  Sparta Global', '  Maidstone Borough Council', '  Screwfix', '  Sherwood Forest Hospitals NHS Foundation Trust', '  Thames Water Utilites', '  Morrisons', '  Sparta Global', '  NHS Scotland', ' Nuclear Decommissioning Authority', '  Centrica', ' DBS Datamarketing Ltd', '  GCHQ', ' Single Homelessness Prevention Service', '  MaritzCX', ' Ito World', ' Expleo Group', '  Sparta Global', ' Black Swan Data Ltd', '  University of Sheffield', '  Capital One - UK', '  Greater London Authority', ' Starcom', ' Attest', '  Wiltshire Health and Care', '  Amazon UK Services Ltd ', '  Away Resorts', ' DBS Datamarketing Ltd', ' Equinox by Work AnyWare', ' trueology', ' Single Homelessness Prevention Service', ' Eurofins UK BioPharma Services', ' Valpak Ltd', '  Halfords', '  Capita Plc', '  Lloyds Banking Group', '  Capita IT Resourcing', '  Citi', '  Amazon UK

In [53]:
ads.loc[(ads['company']=='harnham')]

,company,date,description,extraction_date,full_description,job_title,jobtype,location,other_deets,salary,salary_high,salary_low


In [ ]:
import numpy as np
import datetime

ads = ads.drop(['description', 'other_deets'], axis=1)
ads.columns = ['company', 'job_title',  'salary','location','duration', 'description', 'jobtype','salary_low', 'salary_high']


#industry: which industry sector
    
ads['industry'] = np.nan

#education: required level and subject of education

ads['education'] = np.nan

#**career: **
ads['career'] = np.nan

#duration: How long the job had been advertised for at time of scraping
ads['duration'] = np.nan



# salary average
#salary hour/week/year
# duration 